# Tutorial 03.1 Tensor decomposition

In [19]:
import torch
import numpy as np

sz = [2, 3, 2, 3, 4]  # local space dimensions
T = torch.arange(1, np.prod(sz) + 1, dtype=torch.float).reshape(sz)  # rank-5 tensor
T = T / T.norm()  # normalize

Q = [None] * len(sz)
R = T.clone()  # temporary tensor to be QR-decomposed
szl = 1  # the bond dimension of the left leg of Q[n] to be obtained after
         # the QR decomposition at iteration n; for n = 1, szl = 1 for the dummy leg
for it in range(len(sz) - 1):
    R = R.reshape(szl * sz[it], -1)
    Q[it], R = torch.qr(R)
    Q[it] = Q[it].reshape(szl, sz[it], -1 // szl // sz[it])
    Q[it] = Q[it].permute(0, 2, 1)  # permute to the left-right-bottom order
    szl = Q[it].shape[1]  # update the bond dimension
    R = R.reshape(szl, *sz[it + 1:])

Q[-1] = R.permute(0, 2, 1)
## 
# Now let's contract the tensors |Q{n}| to make a rank-5 tensor again.
# 
# We first remove the first (i.e., left) leg of |Q{1}| which is dummy, either 
# by reshaping |Q{1}| or by permuting the first leg to the end. The latter approach 
# works since PyTorch automatically suppresses trailing singleton dimensions; such 
# permuted leg will not appear explicitly in the array. Also, we want to sort 
# the remaining two legs of |Q{1}| in the bottom (physical)-right order. All these 
# treatment can be done by a single line of |permute|:

T2 = Q[0].permute(1, 2, 0)[1:, :, :]  # remove dummy leg and permute legs

## 
# And we contract tensors.

for it in range(1, len(sz)):
    T2 = T2.matmul(Q[it].permute(0, 2, 1)).permute(0, 2, 1, 3)

## 
# Let's check whether |T2| and |T| are the same.

print(T.shape)
print(T2.shape)
print(torch.max(torch.abs(T - T2)))


RuntimeError: number of dims don't match in permute